# **Reproducing Table 2**

After reproducing Figure 1 of the paper, the Kaplan-Meier survival curves, we began on reproducing the actual outcome of the paper: Table 2. Table 2 gives the model performance using C-index on 20 studied cancer types, using different combinations of data modalities.

Next to the already used clinical data, we had to work with gene expression data (mRNA), miRNA data and whole slide images.

We wanted to start with the most simple combination of the data modalities and extend the code later to also use other modalities as well. We began with the second column of the table using miRNA and clinical data. 

We first wanted to use the author's code for this, to see what outcome we would get. So we tried using the chart1.py code of the authors. 


---




**Mounting Google Drive and importing modules**

Because this notebook is made in Google Colab, the necessary modules are placed in a Google Drive, which had to be mounted to use it in Google Colab. 
That is wat you will see in the next lines of code.
The first reproducability problem already started here as their module fetch.py needs the file: fetch_datachache.npz to work. However, this file was nowhere to be found, so we decided to comment out the modules that needed this file, e.g. generators and data, and we tried writing the code for this ourselves. 

In [0]:
## Mount drive and import all necessary modules
from google.colab import drive
drive.mount('/content/drive')
import os, sys, random, yaml, gc
from itertools import *

import numpy as np
import pandas as pd
import json
import csv
import gzip
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

!pip install lifelines
!pip install xmltodict
!pip install pandas

from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, roc_auc_score
from lifelines.utils import concordance_index
from scipy.stats import pearsonr

from sklearn.model_selection import ShuffleSplit

%cd 'drive/My Drive/Reproducibility_Project/MultimodalPrognosis-master/'

from utils import *
from models import TrainableModel
from modules import Highway
## Following two modules of authors were not usable, so had to be commented out and written by ourselves
# from generators import AbstractPatientGenerator
# from data import fetch

sys.path.append('/content/drive/My Drive/Reproducibility_Project/MultimodalPrognosis-master/')

## Following two modules were written by ourselves and can be found in Github
from one_hot_encoding_clinical_dataset import *

import IPython

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive/Reproducibility_Project/MultimodalPrognosis-master/'
/content/drive/My Drive/Reproducibility_Project/MultimodalPrognosis-master


**Trainable Model**

In the following code cell, the trainable model is defined for miRNA and clinical data. The different layers are implemented as presented in the original paper. The code is the original code, written by the authors of the paper.

In [0]:
#For miRNA and clinical

MULTIMODAL = sys.argv[1]
OUTFILE = f"results/{MULTIMODAL}_mirna_clinical.txt"
print (OUTFILE)

class Network(TrainableModel):

    def __init__(self):
        super(Network, self).__init__()

        self.fcm = nn.Linear(1881, 256)
        self.fcc = nn.Linear(7, 256)
        self.fcg = nn.Linear(60483, 256)
        self.highway = Highway(256, 10, f=F.relu)
        self.fc2 = nn.Linear(256, 2)
        self.fcd = nn.Linear(256, 1)
        self.bn1 = nn.BatchNorm1d(256)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(1, affine=True)

    def forward(self, data, mask):

        x = data['mirna']
        x = x.view(x.shape[0], -1)
        x = F.dropout(x, 0.4)
        x = F.tanh(self.fcm(x))

        y = data['clinical']
        y = y.view(y.shape[0], -1)
        y = F.tanh(self.fcc(y))

        mean = masked_mean((x, y), (mask["mirna"], mask["clinical"]))

        var = masked_variance((x, y), (mask["mirna"], mask["clinical"])).mean()
        var2 = masked_mean (((x - mean.mean())**2, (y - mean.mean())**2), \
                            (mask["mirna"], mask["clinical"]))

        ratios = var/var2.mean(dim=1)
        ratio = ratios.clamp(min=0.02, max=1.0).mean()

        x = mean

        x = self.bn1(x)
        x = F.dropout(x, 0.5, training=self.training)
        x = self.highway(x)
        x = self.bn2(x)

        score = F.log_softmax(self.fc2(x), dim=1)
        hazard = self.fcd(x)

        return {"score": score, "hazard": hazard, "ratio": ratio.unsqueeze(0)}

    def loss(self, pred, target):

        vital_status = target["vital_status"]
        days_to_death = target["days_to_death"]
        hazard = pred["hazard"].squeeze()

        loss = F.nll_loss(pred["score"], vital_status)

        _, idx = torch.sort(days_to_death)
        hazard_probs = F.softmax(hazard[idx].squeeze()[1-vital_status.byte()])
        hazard_cum = torch.stack([torch.tensor(0.0)] + list(accumulate(hazard_probs)))
        N = hazard_probs.shape[0]
        weights_cum = torch.range(1, N)
        p, q = hazard_cum[1:], 1-hazard_cum[:-1]
        w1, w2 = weights_cum, N - weights_cum

        probs = torch.stack([p, q], dim=1)
        logits = torch.log(probs)
        ll1 = (F.nll_loss(logits, torch.zeros(N).long(), reduce=False) * w1)/N
        ll2 = (F.nll_loss(logits, torch.ones(N).long(), reduce=False) * w2)/N
        loss2 = torch.mean(ll1 + ll2)

        loss3 = pred["ratio"].mean()
        
        return loss + loss2 + loss3*0.3

    def score(self, pred, target):
        #R, p = pearsonr(pred, target)
        vital_status = target["vital_status"]
        days_to_death = target["days_to_death"]
        score_pred = pred["score"][:, 1]
        hazard = pred["hazard"][:, 0]

        auc = roc_auc_score(vital_status, score_pred)
        cscore = concordance_index(days_to_death, -hazard, np.logical_not(vital_status))

        return {"AUC": auc, "C-index": cscore, "Ratio": pred["ratio"].mean()}


**Generating data**

As previously mentioned the data generator did not work because of the missing file. Commented out you can see the code as it was originally written by the authors. 

We tried writing our own code to load the data and process it in such a way that it will eventually be a tensor of clinical data, miRNA data, vital status and days to death. The same format as we thought their data looked like before training. 

It was really hard figuring out how everything was put together, because there are a lot of different files with different modules the authors originally used.

Another problem was working with the data, because it is so large. In the paper there wasn't a specific description of what the data looked like and in what format they used it for training. So we had to gather that information out of all the unstructured codes.
For the clinical data we eventually ended up with using one-hot-encoding for race and cancer type. We wrote this code ourselves, imported above as one_hot_encoding_clinical_dataset. The code can be found in Github. In this code the cancer types with abbreviations READ & COAD were given the same one-hot-encoding, as they are on the same line in Table 2 of the paper.

In [0]:
## The following code of the authors could not been used, so we had to write it ourselves

# class DataGenerator(AbstractPatientGenerator):

#     def __init_(self, **kwargs):
#         super(DataGenerator, self).__init__(**kwargs)

#     def sample(self, case, mode='train'):

#         mirna_data = fetch.mirna_data(case)

#         if mirna_data is not None:
#             mirna_data = torch.tensor(mirna_data).float()

#         gene_data = fetch.gene_data(case)

#         if gene_data is not None:
#             gene_data = torch.tensor(gene_data).float()

#         clinical_data = fetch.clinical_data_expanded(case)
        
#         if clinical_data is not None:
#             clinical_data = torch.tensor(clinical_data).float()

#         vital_status = fetch.vital_status(case)
#         days_to_death = fetch.days_to_death(case)
#         if days_to_death is False or days_to_death is None:
#             vital_status = True
#             days_to_death = 20000
        
#         if MULTIMODAL == 'multi':
#             if mode == 'train' and random.randint(1, 4) == 1: clinical_data = None
#             if mode == 'train' and random.randint(1, 4) == 1: mirna_data = None
#             if mode == 'train' and random.randint(1, 4) == 1: gene_data = None

#         if clinical_data is None and mirna_data is None: return None
#         if vital_status is None: return None

#         return {"clinical": clinical_data, "mirna": mirna_data},\
#                 {"vital_status": torch.tensor(vital_status).long(),
#                 "days_to_death": torch.tensor(days_to_death).float()}
    

if __name__ == "__main__":

    model = Network()
    model.compile(optim.Adam, lr=8e-4)

#---------
# Own code to generate data

# Clinical data
    cancer_data = pd.read_csv('data/pancancer_biospecimen (1).csv', sep='\t', index_col=False) 
    cases = []
    cancer_types = []
    for row in cancer_data['barcode']:
        case = row.split('-')[0:3]
        case = "-".join(case)
        cases.append(case)
    for row in cancer_data['project']:
        cancer_type = row.split('-')[1]
        cancer_types.append(cancer_type)

    with open('data/original_clinical_dataset.json') as f:
        clinical = json.load(f)

    clinical_data = np.empty((len(clinical),43),).astype(float)
    ID_ref = np.empty(len(clinical)).astype(object)
    vital_status = np.empty((len(clinical),1))
    days_to_death = np.empty((len(clinical),1))
    j=0
    for line in clinical:
      err = 0
      if 'diagnoses' in line:
        ID = line['diagnoses'][0]['submitter_id'].split("_")[0]
        ID_ref[j] = ID
        try: 
          index = cases.index(ID)
        except ValueError: ## Some patients were not present in biospecimen file. This surpresses error message 
          err = 1

        if err == 1:
          clinical_data[j] = np.concatenate((np.array([None]),np.array([None]),np.zeros(5),np.zeros(36)),axis=0)
        else:
          disease = cancer_types[index]
          keys = ['gender','age_at_index','race']
          if not all(key in line['demographic'] for key in keys): clinical_data[j] = np.concatenate((np.array([None]),np.array([None]),np.zeros(5),np.zeros(36)),axis=0)
          clinical_data[j] = np.concatenate((encoding_gender(line['demographic']['gender']),np.array([line['demographic']['age_at_index']]),encoding_race(line['demographic']['race']),encoding_cancer_type(disease)),axis=0)
          vital_status[j] = encoding_vital_status(line['demographic']['vital_status'])
          if 'days_to_death' in line['demographic']:
            days_to_death[j] = line['demographic']['days_to_death']
          else:
            days_to_death[j] = 20000
            
      else:
        clinical_data[j] = np.concatenate((np.array([None]),np.array([None]),np.zeros(5),np.zeros(36)),axis=0)
      j+=1
    
#MiRNA data
    with gzip.open('data/output/miRNA/colorectal_READ_miRNA_data.pkl','rb') as f:
        miRNA_out = pickle.load(f)

    miRNA_data = np.empty((len(clinical),1881))
    for line in list(miRNA_out.index.values):
        ID = "-".join(line.split('-')[0:3])
        indx = np.where(ID_ref == ID)
        miRNA_data[indx] = miRNA_out.loc[line] 


**Split in train and test data**

The paper mentioned that the dataset of 11 160 patients was split into training and test datasets in 85/15 ratio, stratified by cancer type in order to ensure the same distribution of cancers in both the training and the test set. 

We implemented a code for this ourselves.


In [0]:
## Generate train and test data 

    #Find train and test indices using cancer_types to guarantee same distribution per cancer type
        
    cancer_types = np.arange(0,36)
    data = clinical_data
    test_size = 0.15
    training_size = 0.85

    train_index = []
    test_index = []
    for i in cancer_types:
      cancer_type = np.zeros((1,data.shape[1]))
      cancer_type = np.vstack((cancer_type, data[data[:,7+i] == 1]))
      cancer_type = np.delete(cancer_type,0,axis=0)
      if len(cancer_type)==0:
        pass
      else:
        rs = ShuffleSplit(1, test_size, training_size)
        for train_index_new, test_index_new in rs.split(cancer_type):
          train_index = np.append(train_index,train_index_new)
          test_index = np.append(test_index,test_index_new)
  
    trn_idx = train_index
    tst_idx = test_index
    
  #trn_idx, tst_idx = stratify(clinical_data,np.arange(0,36),0.15, 0.85)

    # Train data
    train = np.empty((len(trn_idx),2)).astype(object)
    i=0
    for index in trn_idx: 
        clinical_train = clinical_data[int(index)]
        miRNA_train = miRNA_data[int(index)]
        vital_status_train = vital_status[int(index)]
        days_to_death_train = days_to_death[int(index)]

        train[i][0] = {"clinical": torch.tensor(clinical_train).float(), "mirna": torch.tensor(miRNA_train).float()}
        train[i][1] = {"vital_status": torch.tensor(vital_status_train).long(), "days_to_death": torch.tensor(days_to_death_train).float()}
        i+=1

    # Test data
    test = np.empty((len(tst_idx),2)).astype(object)
    j=0
    for index in tst_idx: 
        clinical_test = clinical_data[int(index)]
        miRNA_test = miRNA_data[int(index)]
        vital_status_test = vital_status[int(index)]
        days_to_death_test = days_to_death[int(index)]

        test[j][0] = {"clinical": torch.tensor(clinical_test).float(), "mirna": torch.tensor(miRNA_test).float()}
        test[j][1] = {"vital_status": torch.tensor(vital_status_test).long(), "days_to_death": torch.tensor(days_to_death_test).float()}
        j+=1

#------


**Training the data**

In the author's code they eventually end up with for every case an array of dictionairies. But because we took all the cases together, we ended up with a dictionairy of arrays. We thought this wasn't really a problem, because the authors use a stack function to make a dictionary of arrays out of arrays of dictionaries. And as our generated data is already a dictionary of arrays, we decided to remove this function.

We were optimistic and thought if having the data in the right format and splitting it in train and test, the code for training the model would do the job. 

Unfortunately, trying the author's code on our train and test data gave errors instead of results. 

 

In [0]:
# Author's code

    #datagen = DataGenerator(samples=40000, val_samples=10000)

    for epochs in range(0, 5):
        train_data = batched(train, batch_size=64)
        val_data = batched(test, batch_size=64)
        model.fit(train_data, validation=val_data, verbose=True) ## Error occurs in training of the model

        stratified = {}
        for case in datagen.train_cases:
            cancer_type = fetch.cancer_type(case)
            cancer_type = fetch.disease_lookup[cancer_type]
            if cancer_type is None: continue
            stratified.setdefault(cancer_type, []).append(case)

        with open(OUTFILE, "w") as outfile:
            for cancer_type, cases in sorted(stratified.items()):
                print (cancer_type, len(cases))
                # if len(cases) < 64: continue
                test_data = batched(datagen.data(mode='val', cases=cases), batch_size=64)

                try:
                    score = model.eval_data(test_data)
                except:
                    score = 0.0
                print (f"{cancer_type}, {score}", file=outfile)

    model.save("results/predict11.pth") 

 **Not reproducible using the author's code**

After the time we invested in rewriting the code to generate the data in the right format. And finding out their code for training the data still gives no result, we can conclude Table 2 is not reproducible using the code of the authors. And we only tried the modality combination of clinical and miRNA data for one cancer type yet. Imagine using all data modalities for every 20 cancer types. 

Working with the whole slide images wasn't even possible for us, because of the size.

